In [1]:
import time
import numpy as np
from openmm import app
import openmm as mm
from openmm import unit as u
import pdbfixer
from sys import stdout

In [2]:
STRIDE = 100
simulations_steps = 1000
temperature = 300 * u.kelvin
platform_name = 'CPU' #'OpenCL'#
real_time_STRIDE = 1000
friction_coeff = 1.0/u.picoseconds
time_step = 2.0*u.femtoseconds

In [3]:
# Loading the file into the notebook.
pdb_file = 'data/peptide_fixed.pdb'
fixer = pdbfixer.PDBFixer(pdb_file)

In [4]:
# Defining the forcfield and the model
forcefield = app.ForceField('amber14-all.xml', 'amber14/tip3pfb.xml')
model = app.Modeller(fixer.topology, fixer.positions)
model.addSolvent(forcefield,
             padding=1.*u.nanometer,
             model='tip3p', 
             ionicStrength=0.1 * u.millimolar, 
             positiveIon='Na+',
             negativeIon='Cl-',
             neutralize=True )

system = forcefield.createSystem(model.topology, 
                                 nonbondedMethod=app.PME, 
                                 nonbondedCutoff=1*u.nanometers, 
                                 constraints=app.HBonds, 
                                 rigidWater=True, 
                                 ewaldErrorTolerance=0.0005)

integrator = mm.LangevinIntegrator(temperature,
                                   friction_coeff,
                                   time_step)
integrator.setConstraintTolerance(0.00001)

platform = mm.Platform.getPlatformByName(platform_name)
simulation = app.Simulation(model.topology, system, integrator, platform)
simulation.context.setPositions(model.positions)

start = time.time()
simulation.reporters.append(mm.app.pdbreporter.PDBReporter('data/output.pdb', STRIDE))
simulation.reporters.append(mm.app.statedatareporter.StateDataReporter('data/mm.log', STRIDE, step=True, potentialEnergy=True, temperature=True, time=True))
simulation.reporters.append(mm.app.statedatareporter.StateDataReporter(stdout, real_time_STRIDE, step=True, potentialEnergy=True, temperature=True, speed=True, elapsedTime=True))
simulation.minimizeEnergy()
simulation.step(simulations_steps)
end = time.time()

print(f"Elapsed real time: {end-start} s")
print(f'I could do {(60*60)/(end-start)*simulations_steps} simulation steps per hour with {simulations_steps/STRIDE} frames saved.')

#"Step","Potential Energy (kJ/mole)","Temperature (K)","Speed (ns/day)","Elapsed Time (s)"
1000,-133739.40244884774,251.55447429153338,--,0.0
Elapsed real time: 24.91611957550049 s
I could do 144484.77777975536 simulation steps per hour with 10.0 frames saved.
